In [ ]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from transformers import ViTMAEForPreTraining, AutoImageProcessor

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Image Processor

In [ ]:
image_processor_checkpoint = ""
image_processor = AutoImageProcessor.from_pretrained(image_processor_checkpoint)

# Dataset

In [ ]:
class BubblesDataset(Dataset):
    def __init__(self, images_dir, image_processor, transform=None):
        self.images_dir = images_dir
        self.image_processor = image_processor
        self.transform = transform
        
        self.image_list = os.listdir(self.images_dir)
        
        self.imagenet_mean = np.array([0.485, 0.456, 0.406])
        self.imagenet_std = np.array([0.229, 0.224, 0.225])
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, item):
        image = Image.open(self.image_list[0])
        to_tensor = transforms.ToTensor()
        image = to_tensor(image)
        
        image = (image - self.imagenet_mean) / self.imagenet_std
        
        if self.transform:
            image = self.transform(image)
            
        inputs = self.image_processor(image, return_tensors="pt")
        
        return inputs, image

# Model

In [1]:
model_checkpoint = ""
model = ViTMAEForPreTraining.from_pretrained(model_checkpoint)

NameError: name 'ViTMAEForPreTraining' is not defined

In [ ]:
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train(model, optim, criterion, train_dataloader, val_dataloader, epochs_num=1, device='cpu'):
    for epoch in range(epochs_num):
        for inputs in tqdm(train_dataloader):
             pass